In [1]:
import csv
import requests
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [44]:
df = pd.DataFrame()
# Get a list of dictionaries for the injuries 
injuries_data = []
for i in range(0,723): # number of page increments plus one
    url_string = "http://www.prosportstransactions.com/baseball/Search/SearchResults.php?Player=&Team=&BeginDate=1999-03-01&EndDate=2017-11-01&DLChkBx=yes&submit=Search&start="+str(25* i )
    req = requests.get(url_string)
    soup = BeautifulSoup(req.content, 'lxml')
    for item in soup.find_all("tr", {"align":"left"}):# Code for each individual page to capture data
        raw_text = item.text.strip().split("\n")
        injuries_data.append(raw_text)

# Create a dataframe from the injuries data for 723 pages, with 25 per page = 18075 ish        
df = pd.DataFrame(injuries_data)
df.head()

,0,1,2,3,4
0,1999-04-05,Cardinals,,• David Howard,placed on 15-day DL
1,1999-04-05,Cardinals,,• Ray Lankford,placed on 15-day DL
2,1999-04-05,Giants,,• Bill Mueller,placed on 15-day DL
3,1999-04-08,Padres,,• George Arias,placed on 15-day DL
4,1999-04-12,Blue Jays,• Robert Person,,activated from 15-day DL


In [46]:
df.columns = ['Date','Team','Acquired','Relinquished','Notes']
df.head()

,Date,Team,Acquired,Relinquished,Notes
0,1999-04-05,Cardinals,,• David Howard,placed on 15-day DL
1,1999-04-05,Cardinals,,• Ray Lankford,placed on 15-day DL
2,1999-04-05,Giants,,• Bill Mueller,placed on 15-day DL
3,1999-04-08,Padres,,• George Arias,placed on 15-day DL
4,1999-04-12,Blue Jays,• Robert Person,,activated from 15-day DL


In [109]:
df.shape[0]

18066

In [57]:
# Create a dummy column that is 1 if the row represents an injury 
# or a 0 if the row represents a player reactivated.
df['Injury'] = [1 if 'placed' in text else 0 for text in df.Notes]

In [150]:
# Start to extract the number out of the Notes column.
# Replace the hyphen in '15-day' with a space to help splitting and extracting digits.
df.Notes = df.Notes.apply(lambda x: x.replace('-',' '))

In [185]:
def filter_notes_for_DL(notes):
    if '15' in notes:
        return 15
    elif '60' in notes:
        return 60
    elif '10' in notes:
        return 10
    elif '7' in notes:
        return 7
    elif 'restricted' in notes:
        return 0
    elif 'temporary' in notes:
        return 0
    else:
        return 0

In [186]:
df['DL_length'] = df.Notes.map(filter_notes_for_DL)

In [212]:
def extract_injury(notes):
    """Function parses notes column
    to obtain the injury type and returns a string"""
    if len(notes.split('with')) > 1:
        return notes.split('with')[1]
    else:
        return 'unknown'
  

In [200]:
df.Notes.head()

0          placed on 15 day DL
1          placed on 15 day DL
2          placed on 15 day DL
3          placed on 15 day DL
4     activated from 15 day DL
Name: Notes, dtype: object

In [214]:
# Create a column that describes the type of injury based on the notes column using
# the function I created: extract_injury, df['Injury_Type']
df['Injury_Type'] = df.Notes.map(extract_injury)

In [217]:
# What kind of injuries are we looking at?
df['Injury_Type'].value_counts()

unknown                                                                  9627
 strained left hamstring                                                  283
 strained right hamstring                                                 229
 right shoulder inflammation                                              222
 concussion                                                               185
 strained right shoulder                                                  176
 strained left oblique                                                    151
 right elbow inflammation                                                 145
 strained lower back                                                      137
 strained right groin                                                     125
 right shoulder tendinitis                                                117
 strained right elbow                                                     103
 strained left shoulder                                         

## At this point, I have a data frame of all the injury data from the 1999-2000 season to the 2016-2017 season (18 seasons).

# Next Steps:
* Remove all the 'reactivated' rows so that just the 'placed on the DL' are left, since I'm interested most of all in the data on getting injured.
* My one concern is that not knowing when someone comes back might be annoying. Do I really want to re-grab that data later, when I look at player data?

In [219]:
# Remove rows where df['Injury']==0
print('Before removing reactivations:',df.shape)
df = df[df.Injury != 0]
print('With only placements onto the Disabled List:',df.shape)

Before removing reactivations: (18066, 8)
With only placements onto the Disabled List: (8346, 8)


In [220]:
df.to_csv('injuries.csv',index=False)